In [1]:
import gensim
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from sklearn.datasets import fetch_20newsgroups
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.corpus import stopwords
from sklearn import preprocessing
import nltk
import re
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report

C:\Users\Arfat\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
class Classify20:
    def avgfolds(self,txt):
        x1=pd.read_csv(txt+"0.csv")
        x2=pd.read_csv(txt+"1.csv")
        x3=pd.read_csv(txt+"2.csv")
        X=pd.concat((x1,x2,x3))
        by_row_index = X.groupby(X.index)
        df_means = by_row_index.mean()
        n=df_means.round(2)
        n.insert(loc=0,column='Class',value=self.twenty_all.target_names)
        #n=n.drop('Support',axis=1)
        n.loc['20']=n.mean()
        n.loc['20','Class']='avg'
        n=n.round(2)
        n.to_csv(txt+'consolidate.csv',index = False)

    def reporttocsv(self,txt,i):
        report_data = []
        lines = self.report.split('\n')
        x=0
        for line in lines[2:-3]:
            row = {}
            d=re.findall('(\d+(\.\d{1,2})?|\d+)', line)
            row['Class'] = self.twenty_all.target_names[x]
            row['Precision'] = d[0][0]
            row['Recall'] = d[1][0]
            row['F1_score'] = d[2][0]
            row['Support'] = d[3][0]
            report_data.append(row)
            x=x+1
        dataframe = pd.DataFrame.from_dict(report_data)
        dataframe.to_csv(txt+str(i)+'.csv', index = False)
    def myTokenizer(self,text):
        return nltk.regexp_tokenize(text, "\\b[a-zA-Z]{3,}\\b")
    def tokenizeContents(self,contents):
        li=re.findall(self.regex,contents)
        return [self.myTokenizer(content) for content in li]
    def preprocess(self):
        self.model = gensim.models.KeyedVectors.load_word2vec_format('C:/Users/Arfat/Downloads/Compressed/GoogleNews-vectors-negative300.bin', binary=True)
        self.twenty_all = fetch_20newsgroups(subset='all', shuffle=True,remove=('headers', 'footers', 'quotes'))
        full1=[" ".join(data.split("\n")) for data in self.twenty_all.data]
        stopWords = set(stopwords.words('english'))
        self.full= []
        tmp=[] 
        for d in range(len(full1)):
            words = word_tokenize(full1[d])
            for w in words:
                if w not in stopWords:
                    tmp.append(w)
            x=" ".join(tmp)
            tmp=self.myTokenizer(x)
            x=" ".join(tmp)
            tmp=[]
            self.full.append(x)
        self.skf = StratifiedKFold(n_splits=3,shuffle=True,random_state=1)
        y=self.twenty_all.target
        self.regex = r'\b\w+\b'
        self.readydocs()
    def readydocs(self):
        add=np.zeros(300,dtype='float32')
        self.doc=np.zeros((18846,300))
        self.l=[]#for empty documents
        count=0 #total times the key error is raised
        c_names=[] #contains words that are in documents but they don't have vector representations
        count_word_exists=0 #checks number of words having vector representations in the current document
        self.del_index=[] # has index of the documnets where no word has a vector representation

        rate=[] #hit rate for words having vector representations in a document
        for q1 in range(len(self.full)):
            token=self.tokenizeContents(self.full[q1])
            token = list(filter(None, token))
            if len(token)==0:
                self.l.append(q1)
                rate.append(0)
                self.doc[q1]=np.zeros(300,dtype='float32')
                continue
            for q in range(len(token)):
                try:
                    add=add+self.model.word_vec("".join(token[q]))
                    count_word_exists=count_word_exists+1

                except KeyError:
                    count+=1
                    if "".join(token[q]) not in c_names:
                        c_names.append("".join(token[q]))
                    continue
        #print(add,len(token))
            if count_word_exists==0:
                self.del_index.append(q1)
                add=np.zeros(300,dtype='float32')
                self.doc[q1]=add
                rate.append(0)
                continue
            rate.append(count_word_exists/len(token))
            add=add/count_word_exists
            self.doc[q1]=add
            add=np.zeros(300,dtype='float32')
            count_word_exists=0
        self.doc=[ value for (i, value) in enumerate(self.doc) if i not in set(self.l+self.del_index) ]
        self.doc=np.asarray(self.doc)
        self.delundocs()###delete unnecessary docs
    def delundocs(self):
        self.y=self.twenty_all.target                                 ##############deleting docs for all methods
        self.full=[ value for (i, value) in enumerate(self.full) if i not in set(self.l+self.del_index) ]
        for i in sorted(set(self.l+self.del_index),reverse=True):
            self.y=np.delete(self.y,i)
    def w2v(self):
    ##########W2V
        clf=LinearSVC()
        for k,(train_index, test_index) in enumerate(self.skf.split(self.doc, self.y)):
            clf.fit(self.doc[train_index],self.y[train_index])
            y_pred=clf.predict(self.doc[test_index])
            print("Fold :",k)
            self.report=classification_report(self.y[test_index],y_pred,target_names=self.twenty_all.target_names)
            self.reporttocsv('w2v',i=k)
        self.avgfolds('w2v')
    def tfidf(self):
    ##################              TF-IDF
        tfvec=TfidfVectorizer(ngram_range=(1,2))
        X=tfvec.fit_transform(self.full)
        clf = MultinomialNB()
        for k,(train_index, test_index) in enumerate(self.skf.split(X, self.y)):

            clf.fit(X[train_index],self.y[train_index])
            y_pred=clf.predict(X[test_index])
            print("Fold :",k)
            self.report=classification_report(self.y[test_index],y_pred,target_names=self.twenty_all.target_names)
            self.reporttocsv('tfidf',i=k)
        self.avgfolds('tfidf')
    def ftext(self,cmd):
    ##################              Fast text
        cmd1="C:/Users/Arfat/Downloads/Compressed/amazonreviews/fasttext supervised "
        cmd1=cmd1+cmd
        cmd1=cmd1+" -input train20.txt -output model20 >> ftrainresult.txt"
        #print (cmd1)
        #return
        cmd2="C:/Users/Arfat/Downloads/Compressed/amazonreviews/fasttext test model20.bin test20.txt >> ftestresult.txt"
        cmd3="C:/Users/Arfat/Downloads/Compressed/amazonreviews/fasttext predict model20.bin test20.txt > result20.txt"
        c=0
        for k,(train_index, test_index) in enumerate(self.skf.split(self.full, self.y)):
            outF = open("train20.txt", "w")
            out1 = open("test20.txt", "w")
            for i in range(len(train_index)):
                outF.write("__label__")
                outF.write(str(self.y[train_index[i]]))
                outF.write(" ")
                try :
                    outF.write(self.full[train_index[i]])
                    outF.write("\n")
                except UnicodeEncodeError:
                    print("i",i)
                    outF.write("\n")
                    continue
            os.system(cmd1)
            print(len(test_index))
            c=0
            for j in range(len(test_index)):
                out1.write("__label__")
                out1.write(str(self.y[test_index[j]]))
                out1.write(" ")
                c=c+1
                try :
                    out1.write(self.full[test_index[j]])
                    out1.write("\n")
                except UnicodeEncodeError:
                    print("j",j)
                    out1.write("\n")
                    continue
            outF.close()
            out1.close()
            os.system(cmd2)
            os.system(cmd3)
            f = open('result20.txt', 'r')
            lines = np.asarray([np.int32("".join(re.findall('\d+', line))) for line in f.readlines()])
            f.close()
            self.report=classification_report(self.y[test_index],lines,target_names=self.twenty_all.target_names)
            self.reporttocsv('fasttext',i=k)
        self.avgfolds('fasttext')
    def wtw2v(self): ####TFIDF weighted w2v
        tfvec=TfidfVectorizer()
        X=tfvec.fit_transform(self.full)
        fn=tfvec.get_feature_names()
        self.docvec=np.zeros((18298,300))
        tx=[]
        for doc in range(len(self.full)):
            add=np.zeros(300,dtype='float32')
            totalwt=0
            feature_index = X[doc,:].nonzero()[1]
            #print (feature_index)
            tfidf_scores = zip(feature_index, [X[doc, x] for x in feature_index])
            for w, s in [(fn[i], s) for (i, s) in tfidf_scores]:
                totalwt=totalwt+round(s,2)
                #print (w, s)
                try: 
                    add=add+(round(s,2)*self.model.word_vec(w))
                except KeyError:
                    #totalwt=totalwt-round(s,2)
                    continue
            try:
                self.docvec[doc]=add/totalwt
                tx.append(totalwt)
            except ZeroDivisionError:
                print(doc,totalwt)
                continue
        #print(self.docvec)
        #print(tx)
        clf=LinearSVC()
        for k,(train_index, test_index) in enumerate(self.skf.split(self.docvec, self.y)):
            clf.fit(self.docvec[train_index],self.y[train_index])
            y_pred=clf.predict(self.docvec[test_index])
            print("Fold :",k)
            self.report=classification_report(self.y[test_index],y_pred,target_names=self.twenty_all.target_names)
            self.reporttocsv('wtw2v',i=k)
        self.avgfolds('wtw2v')

In [3]:
c20=Classify20()

In [4]:
c20.preprocess()

In [5]:
c20.tfidf()

Fold : 0
Fold : 1
Fold : 2


In [6]:
c20.w2v()

Fold : 0
Fold : 1
Fold : 2


In [7]:
c20.ftext("-dim 100 -epoch 100")


6105
6101
6092


In [8]:
c20.wtw2v()

Fold : 0
Fold : 1
Fold : 2
